# ACXF Demo: Adaptive Context-Aware Explanation Generation

This notebook demonstrates the ACXF system for generating adaptive explanations for tabular data classification.


In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import importlib

# Add parent directory to path so src can be imported as a package
sys.path.insert(0, str(Path('..')))

from src.utils.loaders import load_dataset
from src.utils.preprocessing import preprocess_data
from src.models.train_models import train_model
from src.profiling.user_profiler import UserProfiler, UserCategory
from src.explanation.context_engine import ContextAwareEngine, DecisionCriticality, TimePressure
from src.consistency.consistency_tracker import ConsistencyTracker
from src.interface.novice_view import NoviceView
from src.interface.intermediate_view import IntermediateView
from src.interface.expert_view import ExpertView

# Force reload modules to ensure latest code is used (do this after imports)
if 'src.explanation.lime_explainer' in sys.modules:
    importlib.reload(sys.modules['src.explanation.lime_explainer'])
if 'src.explanation.context_engine' in sys.modules:
    importlib.reload(sys.modules['src.explanation.context_engine'])
    # Re-import after reload
    from src.explanation.context_engine import ContextAwareEngine, DecisionCriticality, TimePressure

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## 1. Load and Preprocess Data


In [2]:
# Try to load a dataset
data_dir = Path('..') / 'data'
datasets = {
    'german_credit': data_dir / 'german_credit.csv',
    'diabetes': data_dir / 'diabetes.csv',
    'telco_churn': data_dir / 'telco_churn.csv'
}

# Find available dataset
dataset_name = None
dataset_path = None

for name, path in datasets.items():
    if path.exists():
        dataset_name = name
        dataset_path = path
        break

if dataset_path is None:
    # Create synthetic dataset
    from sklearn.datasets import make_classification
    X, y = make_classification(
        n_samples=1000, n_features=10, n_informative=5,
        n_redundant=2, n_classes=2, random_state=42
    )
    feature_names = [f"Feature_{i}" for i in range(X.shape[1])]
    X_df = pd.DataFrame(X, columns=feature_names)
    y_series = pd.Series(y, name='target')
    print("Using synthetic dataset")
else:
    print(f"Loading dataset: {dataset_name}")
    X_df, y_series, feature_names = load_dataset(dataset_name, str(dataset_path))

print(f"Dataset shape: {X_df.shape}")


Using synthetic dataset
Dataset shape: (1000, 10)


In [3]:
# Preprocess data
X_processed, y_processed, preprocess_info = preprocess_data(X_df, y_series)
feature_names_processed = preprocess_info['feature_names']
print(f"Preprocessed shape: {X_processed.shape}")
print(f"Number of features: {len(feature_names_processed)}")


Preprocessed shape: (1000, 10)
Number of features: 10


## 2. Train Model


In [4]:
# Train model
model, X_train, X_test, y_train, y_test, metrics = train_model(
    X_processed, y_processed, model_type='random_forest', test_size=0.2
)

print(f"Model trained successfully!")
print(f"Train Accuracy: {metrics['train_accuracy']:.4f}")
print(f"Test Accuracy: {metrics['test_accuracy']:.4f}")


Model trained successfully!
Train Accuracy: 1.0000
Test Accuracy: 0.9350


## 3. Initialize ACXF Components


In [5]:
# Initialize consistency tracker
consistency_tracker = ConsistencyTracker(n_clusters=10)
consistency_tracker.fit_clusters(X_train)

# Create user personas
novice_profiler = UserProfiler(UserCategory.NOVICE)
novice_profiler.update_from_questionnaire(1, 2, 2, 2)

intermediate_profiler = UserProfiler(UserCategory.INTERMEDIATE)
intermediate_profiler.update_from_questionnaire(3, 3, 3, 3)

expert_profiler = UserProfiler(UserCategory.EXPERT)
expert_profiler.update_from_questionnaire(5, 5, 4, 5)

# Create views
novice_view = NoviceView()
intermediate_view = IntermediateView()
expert_view = ExpertView()

print("ACXF components initialized!")


ACXF components initialized!


## 4. Generate Adaptive Explanations


In [6]:
# Select a test instance
test_idx = 0
test_instance = X_test[test_idx]
true_label = y_test[test_idx]
prediction = model.predict_proba(test_instance.reshape(1, -1))[0]
predicted_class = np.argmax(prediction)

print(f"Test Instance #{test_idx}")
print(f"True Label: {true_label}, Predicted: {predicted_class}")
print(f"Prediction Probabilities: {prediction}")


Test Instance #0
True Label: 1, Predicted: 1
Prediction Probabilities: [0.04354062 0.95645938]


### 4.1 Novice User Explanation


In [7]:
# Create context engine for novice
novice_engine = ContextAwareEngine(
    model, X_train, feature_names_processed, novice_profiler
)

# Generate explanation
novice_explanation = novice_engine.generate_explanation(
    test_instance,
    decision_criticality=DecisionCriticality.MEDIUM,
    time_pressure=TimePressure.NONE
)

# Render
print("NOVICE USER EXPLANATION")
print("=" * 70)
print(novice_view.render(novice_explanation))

# Visualize
Path('../experiments/plots').mkdir(parents=True, exist_ok=True)
novice_view.visualize(novice_explanation, save_path='../experiments/plots/novice_explanation.png')


NOVICE USER EXPLANATION
The model's decision is primarily influenced by: Feature_4, Feature_5, Feature_0.


### 4.2 Intermediate User Explanation


In [8]:
# Create context engine for intermediate
intermediate_engine = ContextAwareEngine(
    model, X_train, feature_names_processed, intermediate_profiler
)

# Generate explanation
intermediate_explanation = intermediate_engine.generate_explanation(
    test_instance,
    decision_criticality=DecisionCriticality.MEDIUM,
    time_pressure=TimePressure.NONE
)

# Render
print("INTERMEDIATE USER EXPLANATION")
print("=" * 70)
print(intermediate_view.render(intermediate_explanation))

# Visualize
intermediate_view.visualize(intermediate_explanation, save_path='../experiments/plots/intermediate_explanation.png')


INTERMEDIATE USER EXPLANATION
Feature Contribution Analysis:

Method: LIME

 1. Feature_4                      ↓ 0.1427
 2. Feature_5                      ↓ 0.0906
 3. Feature_0                      ↑ 0.0843
 4. Feature_3                      ↑ 0.0592
 5. Feature_9                      ↑ 0.0514
 6. Feature_6                      ↓ 0.0059
 7. Feature_7                      ↓ 0.0033
 8. Feature_1                      ↓ 0.0029
 9. Feature_8                      ↓ 0.0014
10. Feature_2                      ↑ 0.0009

Predicted probabilities:
  Class 0: 0.044
  Class 1: 0.956



### 4.3 Expert User Explanation


In [9]:
# Create context engine for expert
expert_engine = ContextAwareEngine(
    model, X_train, feature_names_processed, expert_profiler
)

# Generate explanation
expert_explanation = expert_engine.generate_explanation(
    test_instance,
    decision_criticality=DecisionCriticality.HIGH,
    time_pressure=TimePressure.NONE
)

# Render
print("EXPERT USER EXPLANATION")
print("=" * 70)
print(expert_view.render(expert_explanation))

# Visualize
expert_view.visualize_shap_bar(expert_explanation, save_path='../experiments/plots/expert_explanation.png')


EXPERT USER EXPLANATION
Detailed Explanation Report

Method: SHAP (Local)
Type: local

Feature Contributions:
--------------------------------------------------
  1. Feature_9                             0.074056
  2. Feature_8                            -0.074056
  3. Feature_3                             0.058442
  4. Feature_2                            -0.058442
  5. Feature_1                             0.052908
  6. Feature_0                            -0.052908
  7. Feature_7                            -0.021684
  8. Feature_6                             0.021684
  9. Feature_4                            -0.002275
 10. Feature_5                             0.002275

Base Value (Expected): 0.501037

Predicted Probabilities:
  Class 0: 0.043541
  Class 1: 0.956459
  Predicted Class: 1

Instance Values:
  Feature_0: 0.0700
  Feature_1: -1.9764
  Feature_2: 0.5167
  Feature_3: -0.4833
  Feature_4: -0.6989
  Feature_5: -0.8536
  Feature_6: -1.3213
  Feature_7: 0.5003
  Feature_8: -0.

## 5. Counterfactual Explanations


In [10]:
# Generate counterfactual
counterfactual = expert_engine.generate_counterfactual(test_instance)

print("COUNTERFACTUAL EXPLANATION")
print("=" * 70)
print(f"Original Prediction: {counterfactual.get('original_prediction', [])}")
print(f"Counterfactual Prediction: {counterfactual.get('counterfactual_prediction', [])}")
print(f"Class Changed: {counterfactual.get('class_changed', False)}")

if counterfactual.get('modifications'):
    print("\nModifications:")
    for mod in counterfactual['modifications'][:5]:
        print(f"  {mod.get('feature', 'Unknown')}: "
              f"{mod.get('original_value', 0):.3f} -> {mod.get('new_value', 0):.3f}")

# Visualize
expert_view.visualize_counterfactual(counterfactual, save_path='../experiments/plots/counterfactual.png')


COUNTERFACTUAL EXPLANATION
Original Prediction: [0.0435406162464986, 0.9564593837535013]
Counterfactual Prediction: [0.1531111111111111, 0.8468888888888888]
Class Changed: False

Modifications:
  Feature_4: -0.699 -> -1.048
  Feature_5: -0.854 -> -1.280
  Feature_0: 0.070 -> 0.035
  Feature_3: -0.483 -> -0.242
  Feature_9: 1.350 -> 0.675


## 6. Temporal Consistency


In [11]:
# Test consistency across multiple instances
test_indices = [0, 1, 2, 3, 4]
consistency_scores = []

for idx in test_indices:
    instance = X_test[idx]
    explanation = expert_engine.generate_explanation(instance)
    
    consistency = consistency_tracker.compute_consistency_score(instance, explanation)
    consistency_scores.append(consistency.get('consistency_score', 0.0))
    
    consistency_tracker.add_explanation(instance, explanation)

print(f"Average Consistency Score: {np.mean(consistency_scores):.3f}")
print(f"Consistency Scores: {consistency_scores}")


Average Consistency Score: 0.000
Consistency Scores: [0.0, 0.0, 0.0, 0.0, 0.0]


## 7. Summary

This demo showed:
1. How ACXF adapts explanations based on user expertise level
2. Different explanation formats for novice, intermediate, and expert users
3. Counterfactual explanation generation
4. Temporal consistency tracking

For full evaluation, run the `evaluation_results.ipynb` notebook.
